## Replace entities with placeholder

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model và tokenizer
model_name = "linhdzqua148/xlm-roberta-ner-japanese-railway"
tokenizer_ner = AutoTokenizer.from_pretrained(model_name)
model_ner = AutoModelForTokenClassification.from_pretrained(model_name).to(device)

id_to_label = {0: 'O', 1: 'B-STATION', 2: 'I-STATION', 3: 'B-LINE', 4: 'I-LINE'}

def predict_entities(text_tokens, model, tokenizer):
    model.eval()
    inputs = tokenizer(
        text_tokens,
        return_tensors="pt",
        truncation=True,
        padding=True,
        is_split_into_words=True
    ).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=2)
    word_ids = inputs.word_ids()
    predicted_labels = []
    seen_words = set()
    for i, word_id in enumerate(word_ids):
        if word_id is not None and word_id not in seen_words:
            predicted_labels.append(id_to_label[predictions[0][i].item()])
            seen_words.add(word_id)
    min_len = min(len(text_tokens), len(predicted_labels))
    return list(zip(text_tokens[:min_len], predicted_labels[:min_len]))

/mnt/disk1/aiotlab/envs/linh_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import MeCab

try:
    mecab = MeCab.Tagger("-Owakati") 
    print("✅ MeCab hoạt động")
except Exception as e:
    print("❌ Lỗi:", e)

def improved_tokenize(text):
    tokens = mecab.parse(text).strip().split()
    return tokens

print(improved_tokenize("これはテストです"))

✅ MeCab hoạt động
['これ', 'は', 'テスト', 'です']


In [3]:
import re
def normalize_entity(entity_text):
    """
    Normalize entity: strip suffix + split number
    """
    SUFFIXES_TO_STRIP = ["方面行き", "方面", "行き"]
    NUM_SPLIT_PAT = re.compile(r"^(.+?)(\d+号)$")
    
    for suffix in SUFFIXES_TO_STRIP:
        if entity_text.endswith(suffix):
            entity_text = entity_text[:-len(suffix)]
            break
    
    match = NUM_SPLIT_PAT.match(entity_text)
    if match:
        base, suffix = match.groups()
        return [base, suffix]
    else:
        return [entity_text]

In [4]:
def extract_and_normalize_entities(predicted):
    """
    Trích xuất entities và normalize chúng ngay từ đầu
    """
    entities = []
    i = 0
    
    while i < len(predicted):
        token, label = predicted[i]
        
        if label.startswith("B-"):
            ent_type = label.split("-", 1)[1]
            ent_tokens = [token]
            i += 1
            
            while i < len(predicted) and predicted[i][1] == f"I-{ent_type}":
                ent_tokens.append(predicted[i][0])
                i += 1
            
            entity_text = "".join(ent_tokens)
            
            # Normalize entity ngay tại đây
            normalized_entities = normalize_entity(entity_text)
            entities.extend(normalized_entities)
        else:
            i += 1
    
    return entities

In [5]:
def create_placeholder_mapping(text, entities):
    """
    Tạo mapping giữa entity và placeholder dựa trên vị trí xuất hiện
    """
    offsets = []
    for ent in entities:
        pos = text.find(ent)
        if pos != -1:
            offsets.append((pos, ent))
    
    offsets.sort() 
    
    ph2ent = {}
    ent2ph = {}
    ph_counter = 0
    
    for pos, ent in offsets:
        if ent not in ent2ph:
            ph = f"[PH{ph_counter}]"
            ent2ph[ent] = ph
            ph2ent[ph] = ent
            ph_counter += 1
    
    return ph2ent, ent2ph

In [7]:
def mask_text_with_placeholders(text, ent2ph):
    """
    Thay thế entities trong text bằng placeholders
    """
    result_text = text
    
    sorted_entities = sorted(ent2ph.keys(), key=len, reverse=True)
    
    for ent in sorted_entities:
        result_text = result_text.replace(ent, ent2ph[ent])
    
    return result_text

In [8]:
def replace_entity_and_map(txt: str):
   
    try:
        # Tokenize
        tokens = improved_tokenize(txt)
        
        # Predict entities
        predicted = predict_entities(tokens, model_ner, tokenizer_ner)
        
        # Extract và normalize entities
        normalized_entities = extract_and_normalize_entities(predicted)
        
        # Tạo mapping placeholder
        ph2ent, ent2ph = create_placeholder_mapping(txt, normalized_entities)
        
        # Mask text
        final_text = mask_text_with_placeholders(txt, ent2ph)
        
        return final_text, ph2ent
        
    except Exception as e:
        print(f"❌ Lỗi xử lý: {e}")
        return txt, {}

In [9]:
txt = input()
print(predict_entities( improved_tokenize(txt), model_ner, tokenizer_ner  ))

replace_entity_and_map(txt)

[('ま', 'O'), ('も', 'O'), ('なく', 'O'), ('、', 'B-STATION'), ('品川', 'B-STATION'), ('、', 'B-STATION'), ('品川', 'B-STATION'), ('。', 'O'), ('お', 'O'), ('出口', 'O'), ('は', 'O'), ('右側', 'O'), ('です', 'O'), ('。', 'B-LINE'), ('京浜', 'B-LINE'), ('東北', 'I-LINE'), ('線', 'I-LINE'), ('、', 'B-LINE'), ('山手', 'B-LINE'), ('線', 'I-LINE'), ('、', 'B-LINE'), ('京急', 'B-LINE'), ('線', 'I-LINE'), ('は', 'O'), ('お', 'O'), ('乗り換え', 'O'), ('です', 'O'), ('。', 'O'), ('本日', 'O'), ('も', 'B-LINE'), ('JR', 'B-LINE'), ('東', 'I-LINE'), ('日本', 'I-LINE'), ('を', 'O'), ('ご', 'O'), ('利用', 'O'), ('ください', 'O'), ('まし', 'O'), ('て', 'O'), ('、', 'O'), ('ありがとう', 'O'), ('ござい', 'I-LINE'), ('まし', 'I-LINE'), ('た', 'I-LINE'), ('。', 'O')]


('ま[PH0]なく[PH1][PH2][PH1][PH2][PH3]お出口は右側です[PH3][PH4][PH1][PH5][PH1][PH6]はお乗り換えです[PH3]本日[PH0][PH7]をご利用くださいまして[PH1]ありがとうございました[PH3]',
 {'[PH0]': 'も',
  '[PH1]': '、',
  '[PH2]': '品川',
  '[PH3]': '。',
  '[PH4]': '京浜東北線',
  '[PH5]': '山手線',
  '[PH6]': '京急線',
  '[PH7]': 'JR東日本'})

## dịch máy và dịch thực thể

In [2]:
import json
import pandas as pd
import torch
import re
import jaconv
from fugashi import Tagger
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import requests

/mnt/disk1/aiotlab/envs/linh_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch 

model_name = "linhdzqua148/jrw-mt-ja-en"
device = "cuda" if torch.cuda.is_available() else "cpu" # Check if CUDA is available
try:
    tok = AutoTokenizer.from_pretrained(model_name)
    tokenizer = tok
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    print(f"Model '{model_name}' loaded successfully on {device}.")
except Exception as e:
    print(f"Error loading model: {e}")
    device = "cpu"
    try:
        tok = AutoTokenizer.from_pretrained(model_name)
        tokenizer = tok
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
        print(f"Model loaded on CPU due to error on GPU.")
    except Exception as cpu_e:
        print(f"Error loading model on CPU as well: {cpu_e}")
        tokenizer = None
        model = None
        tagger = None # Disable tagger if model loading fails critically
        print("Translation functionality will be limited.")

/mnt/disk1/aiotlab/envs/linh_env/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Model 'linhdzqua148/opus-mt-ja-en-railway-7' loaded successfully on cuda.


In [22]:
import pandas as pd

def get_entity_mapping_from_csv(file_path):
    """
    Đọc file CSV và tạo dictionary mapping giống như function database
    """
    try:
        print("📁 Đang đọc file CSV...")
        # Đọc file CSV trên Kaggle
        df = pd.read_csv(file_path, encoding='utf-8')
        
        print(f"📊 Đã đọc {len(df)} dòng dữ liệu")
        
        # Loại bỏ dữ liệu null/empty giống như logic database
        df_clean = df.dropna(subset=['kanji', 'english'])
        df_clean = df_clean[(df_clean['kanji'] != '') & (df_clean['english'] != '')]
        
        # Tạo dictionary mapping: kanji -> english
        entity_direct_map_csv = dict(zip(df_clean['kanji'], df_clean['english']))
        
        print(f"✅ Hoàn tất! Tổng số entity: {len(entity_direct_map_csv)}")
        
        return entity_direct_map_csv
        
    except Exception as e:
        print(f"❌ Lỗi: {e}")
        return {}

In [25]:
entity_direct_map_csv = get_entity_mapping_from_csv('./train_entity.csv')
print(f"Dictionary size: {len(entity_direct_map_csv)}")

📁 Đang đọc file CSV...
📊 Đã đọc 19174 dòng dữ liệu
✅ Hoàn tất! Tổng số entity: 18596
Dictionary size: 18596


In [48]:
def get_en_name_from_wikidata(japanese_name):
    """
    Tìm tên tiếng Anh từ Wikidata cho entities đường sắt
    Chỉ xử lý các entities hợp lệ cho ngữ cảnh đường sắt
    """
    # Filter 1: Loại bỏ các trường hợp không hợp lệ
    if not japanese_name or len(japanese_name.strip()) < 2:
        return None
    
    japanese_name = japanese_name.strip()
    
    # Filter 2: Loại bỏ dấu câu, ký tự đơn lẻ và các từ không phải entity
    punctuation_chars = {'。', '、', '，', '．', '！', '？', '：', '；'}
    if japanese_name in punctuation_chars:
        return None
    
    # Filter 3: Loại bỏ các hạt từ (particles) và từ chức năng tiếng Nhật
    japanese_particles = {
        'は', 'を', 'が', 'に', 'で', 'と', 'の', 'か', 'から', 'まで', 'も', 'へ', 'より', 'だけ', 'ばかり',
        'くらい', 'ほど', 'など', 'しか', 'でも', 'だって', 'って', 'なら', 'たら', 'れば', 'けれど',
        'が', 'けど', 'のに', 'ので', 'から', 'ため', 'ように', 'ために', 'として', 'について',
        'によって', 'に関して', 'に対して', 'について', 'として', 'において', 'において'
    }
    if japanese_name in japanese_particles:
        return None
    
    # Filter 4: Loại bỏ các từ đơn lẻ không có ý nghĩa entity
    single_meaningless_chars = {
        'あ', 'い', 'う', 'え', 'お', 'か', 'き', 'く', 'け', 'こ', 'さ', 'し', 'す', 'せ', 'そ',
        'た', 'ち', 'つ', 'て', 'と', 'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ひ', 'ふ', 'へ', 'ほ',
        'ま', 'み', 'む', 'め', 'も', 'や', 'ゆ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん'
    }
    if japanese_name in single_meaningless_chars:
        return None
    
    # Filter 5: Chỉ xử lý nếu có ký tự chỉ định đường sắt
    railway_indicators = ['駅', '線', 'メトロ', 'Metro', '鉄道', '電車', '新幹線', 'JR', '方面', '地下鉄']
    has_railway_indicator = any(indicator in japanese_name for indicator in railway_indicators)
    
    # Filter 6: Kiểm tra độ dài hợp lý cho tên ga/tuyến
    is_reasonable_length = len(japanese_name) >= 2 and len(japanese_name) <= 25
    
    # Chỉ tiếp tục nếu có chỉ định đường sắt VÀ độ dài hợp lý
    if not (has_railway_indicator and is_reasonable_length):
        return None
    
    search_url = "https://www.wikidata.org/w/api.php"
    search_params = {
        "action": "wbsearchentities",
        "language": "ja",
        "format": "json",
        "search": japanese_name,
        "limit": 5 # Get a few results to be safer
    }

    try:
        # Added explicit headers to mimic a browser request, might help with some firewalls/blocks
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(search_url, params=search_params, timeout=8, headers=headers) # Increased timeout slightly
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        results = response.json().get("search", [])

        if not results:
            return None

        # Pick the first result và kiểm tra description
        entity_id = results[0]["id"]
        description = results[0].get("description", "").lower()
        
        # Filter 4: Kiểm tra description có liên quan đến đường sắt không
        railway_keywords = ['station', 'railway', 'train', 'metro', 'line', 'subway', 'transit']
        is_railway_related = any(keyword in description for keyword in railway_keywords) if description else True
        
        # Nếu có description nhưng không liên quan đến đường sắt, bỏ qua
        if description and not is_railway_related and not has_railway_indicator:
            return None

        data_url = f"https://www.wikidata.org/wiki/Special:EntityData/{entity_id}.json"

        response = requests.get(data_url, timeout=8, headers=headers) # Added timeout and headers
        response.raise_for_status()

        entity_data = response.json()

        # Try to get the English label
        en_label = entity_data.get("entities", {}).get(entity_id, {}).get("labels", {}).get("en", {}).get("value")

        if en_label:
             return en_label
        else:
             return None # Found entity but no English label

    except requests.exceptions.Timeout:
        print(f"Wikidata request timed out for '{japanese_name}'.")
        return None
    except requests.exceptions.ConnectionError:
        print(f"Wikidata connection error for '{japanese_name}'. Network issue?")
        return None
    except requests.exceptions.HTTPError as e:
         print(f"Wikidata HTTP error for '{japanese_name}': {e}")
         return None
    except ValueError: # JSON decode error
        print(f"Wikidata response is not valid JSON for '{japanese_name}'.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Wikidata lookup for '{japanese_name}': {e}")
        return None

In [49]:
def translate_entity(jp_entity):
    """
    Translates a Japanese entity name (like station, line, etc.) to English
    using a prioritized approach:
    1. Custom CSV Dictionary lookup (Kanji -> English) - Highest priority for specific names
    2. Wikidata lookup - Good for known proper nouns not in CSV
    """
    jp_entity = jp_entity.strip()
    if not jp_entity:
        return "" 
    
    print(f"Translating entity: '{jp_entity}'") # Debug print

    # 1) Tra từ điển tùy chỉnh (CSV)
    if jp_entity in entity_direct_map_csv:
        print(f" -> Matched in CSV dictionary: {entity_direct_map_csv[jp_entity]}")
        return entity_direct_map_csv[jp_entity]

    # 2) Tra từ Wikidata
    wikidata_result = get_en_name_from_wikidata(jp_entity)
    if wikidata_result:
        print(f" -> Matched in Wikidata: {wikidata_result}")
        return wikidata_result

    # 3) Fallback to machine translation
    return jp_entity


In [28]:
def split_sentences(text):
    """Chia câu theo '。', giữ dấu cuối để ghép lại mạch lạc."""
    # This function seems designed for splitting input text for NMT batching,
    # not directly for entity translation.
    parts = [p.strip() for p in text.split("。") if p.strip()]
    return [p + "。" for p in parts]

def translate_text(text_list, batch_size=16):
    """Dịch list câu Nhật → Anh bằng model.generate() (Batch Processing)"""
    # This function is for batch processing sentences, not used by process_row currently.
    if not isinstance(text_list, list):
        print("Error: translate_text expects a list of strings.")
        return [] # Or raise an error
    if model is None or tokenizer is None:
         print("Error: Model or tokenizer not loaded. Cannot perform batch translation.")
         return ["Error: Translator not available."] * len(text_list)


    results = []
    # Use the global 'tokenizer' variable
    # Note: Splitting by '。' might not be ideal for short railway announcements.
    # Consider if you need this splitting logic here or just pass full sentences.
    # Current code does splitting.

    for i in range(0, len(text_list), batch_size):
        batch_texts = text_list[i : i + batch_size]
        seg_pairs = []
        concat_map = [] # maps segment index back to original batch_texts index
        for idx, txt in enumerate(batch_texts):
            # Assuming split_sentences is appropriate here
            segs = split_sentences(txt)
            seg_pairs.extend(segs)
            concat_map.extend([idx] * len(segs))

        # Handle empty seg_pairs if batch_texts were empty or didn't split
        if not seg_pairs:
            results.extend([""] * len(batch_texts))
            continue

        # Encode all segments
        try:
            inputs = tokenizer(seg_pairs, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
            with torch.no_grad():
                generated = model.generate(
                    **inputs,
                    max_length=128,
                    num_beams=6,
                    length_penalty=0.8,
                    # Add other generation args if needed, matching process_row direct call
                )
            decoded = tokenizer.batch_decode(generated, skip_special_tokens=True)
        except Exception as e:
            print(f"Error during batch NMT translation for batch starting with '{batch_texts[0]}': {e}")

        # Merge segments back into original sentence structure
        merged = [""] * len(batch_texts)
        for seg, idx in zip(decoded, concat_map):
             # Careful merging: add space only if the merged string is not empty
             if merged[idx]:
                 merged[idx] += (" " + seg.strip())
             else:
                 merged[idx] += seg.strip()

        # Final cleanup for merged sentences
        merged = [re.sub(r'\s+([.,!?:;])', r'\1', s).strip() for s in merged] # Punctuation spacing
        merged = [re.sub(r'\s+', ' ', s).strip() for s in merged] # Normalize spaces
        merged = [s.replace(' .', '.') for s in merged] # Fix common space before period


        results.extend(merged)

    return results

In [29]:
def remove_adjacent_duplicate_phrases(text, max_phrase_len=5):
    """
    Loại bỏ các cụm từ (từ 1 đến max_phrase_len từ) lặp liên tiếp nhau trong văn bản.
    Hỗ trợ lặp có hoặc không có dấu phẩy, không phân biệt hoa thường.
    """
    # Loại bỏ khoảng trắng thừa ở dấu phẩy
    text = re.sub(r'\s+,', ',', text)

    # Xử lý cụm từ lặp, giảm dần từ cụm dài nhất về 1 từ
    for n in range(max_phrase_len, 0, -1):
        # Tạo regex động cho cụm n từ lặp liên tiếp (có thể có dấu phẩy)
        # Ví dụ cho n=2: (Tokyo Metro)(,? Tokyo Metro)
        pattern = re.compile(
            r'(\b(?:[\w\-\’ōū]+(?:\s+|, ?)){%d}[\w\-\’ōū]+\b)'    # Nhóm 1: n từ
            r'(,? \1\b)'                                          # Nhóm 2: lặp lại
            % (n-1),
            flags=re.IGNORECASE
        )
        # Lặp để loại tất cả các trường hợp lặp liên tiếp
        while pattern.search(text):
            text = pattern.sub(r'\1', text)
    # Sau đó, xử lý riêng lặp 1 từ với dấu phẩy hoặc không dấu phẩy
    text = re.sub(r'\b(\w+)(,? \1\b)', r'\1', text, flags=re.IGNORECASE)
    # Sửa các trường hợp dấu câu dư, khoảng trắng thừa
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s+([.,;:!?])', r'\1', text)
    return text.strip()

In [35]:
def translate_text_simple(text):
    """
    Dịch văn bản đơn giản bằng machine translation model
    """
    if not text.strip():
        return ""
    
    try:
        inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_length=128,
                num_beams=6,
                length_penalty=0.8,
                early_stopping=True,
                do_sample=False
            )
        result = tokenizer.decode(generated[0], skip_special_tokens=True).strip()
        return result
    except Exception as e:
        print(f"❌ Translation error: {e}")
        return text

In [53]:
def translate_entities_with_fallback(ph2ent):
    """
    Dịch entities với logic:
    1. Check trong CSV trước
    2. Không có thì check bằng hàm Wiki
    3. Không có thì thay ngược lại từ đó thay cho placeholder đã thay trong câu và xóa nó trong placeholder map
    
    Args:
        ph2ent: Dictionary mapping placeholder -> entity (ví dụ: {'[PH0]': 'は', '[PH1]': '大阪'})
    
    Returns:
        tuple: (updated_ph2ent, entities_to_restore)
        - updated_ph2ent: Dictionary đã loại bỏ entities không dịch được
        - entities_to_restore: Dictionary mapping placeholder -> entity để thay ngược lại
    """
    updated_ph2ent = {}
    entities_to_restore = {}
    
    print("🔄 Bắt đầu dịch entities...")
    
    for placeholder, entity in ph2ent.items():
        print(f"\n📝 Xử lý {placeholder} -> '{entity}'")
        
        # Bước 1: Check trong CSV
        if entity in entity_direct_map_csv:
            translated = entity_direct_map_csv[entity]
            updated_ph2ent[placeholder] = translated
            print(f"  ✅ Tìm thấy trong CSV: '{entity}' -> '{translated}'")
            continue
        
        # Bước 2: Check bằng hàm Wiki
        wikidata_result = get_en_name_from_wikidata(entity)
        if wikidata_result:
            updated_ph2ent[placeholder] = wikidata_result
            print(f"  ✅ Tìm thấy trong Wiki: '{entity}' -> '{wikidata_result}'")
            continue
        
        # Bước 3: Không tìm được -> thay ngược lại
        entities_to_restore[placeholder] = entity
        print(f"  ❌ Không tìm được bản dịch cho '{entity}' -> sẽ thay ngược lại")
    
    print(f"\n📊 Kết quả:")
    print(f"  - Entities dịch được: {len(updated_ph2ent)}")
    print(f"  - Entities thay ngược lại: {len(entities_to_restore)}")
    
    return updated_ph2ent, entities_to_restore


In [54]:
def restore_entities_and_translate(text_with_placeholders, entities_to_restore, translated_entities):
    """
    Thay ngược lại các entities không dịch được và dịch câu
    
    Args:
        text_with_placeholders: Câu đã thay placeholder (ví dụ: "この電車[PH0][PH1]行き快速電車です。")
        entities_to_restore: Dictionary mapping placeholder -> entity để thay ngược lại
        translated_entities: Dictionary mapping placeholder -> entity đã dịch
    
    Returns:
        tuple: (final_text, final_ph2ent)
        - final_text: Câu đã thay ngược lại entities không dịch được
        - final_ph2ent: Dictionary chỉ chứa entities đã dịch được
    """
    print("🔄 Thay ngược lại entities không dịch được...")
    
    # Bước 1: Thay ngược lại entities không dịch được
    final_text = text_with_placeholders
    for placeholder, original_entity in entities_to_restore.items():
        final_text = final_text.replace(placeholder, original_entity)
        print(f"  🔄 Thay ngược {placeholder} -> '{original_entity}'")
    
    print(f"📝 Text sau khi thay ngược: {final_text}")
    
    # Bước 2: Dịch câu
    print("🔄 Dịch câu...")
    translated_text = translate_text_simple(final_text)
    print(f"📝 Câu đã dịch: {translated_text}")
    
    # Bước 3: Tạo final mapping chỉ chứa entities đã dịch được
    final_ph2ent = translated_entities.copy()
    
    return final_text, translated_text, final_ph2ent


In [55]:
def merge_translation_with_entities(translated_text, final_ph2ent):
    """
    Ghép lại kết quả dịch với entities đã dịch dựa trên placeholder mapping
    
    Args:
        translated_text: Câu đã dịch (ví dụ: "This train is a rapid train bound for [PH1]. Next is [PH2], [PH2].")
        final_ph2ent: Dictionary mapping placeholder -> entity đã dịch (ví dụ: {'[PH1]': 'Osaka', '[PH2]': 'Kyoto'})
    
    Returns:
        str: Câu cuối cùng đã ghép entities
    """
    print("🔄 Ghép lại kết quả với entities đã dịch...")
    
    final_result = translated_text
    
    # Thay thế các placeholder bằng entities đã dịch
    for placeholder, translated_entity in final_ph2ent.items():
        if placeholder in final_result:
            final_result = final_result.replace(placeholder, translated_entity)
            print(f"  ✅ Thay {placeholder} -> '{translated_entity}'")
    
    # Loại bỏ các placeholder còn lại (nếu có)
    import re
    remaining_placeholders = re.findall(r'\[PH\d+\]', final_result)
    if remaining_placeholders:
        print(f"  ⚠️ Còn lại placeholders: {remaining_placeholders}")
    
    print(f"📝 Kết quả cuối cùng: {final_result}")
    
    return final_result


In [56]:
def translate_with_entity_handling(text):
    """
    Hàm tổng hợp thực hiện toàn bộ flow dịch với xử lý entities:
    
    1. Thay thế entities bằng placeholders
    2. Dịch entities (check CSV -> Wiki -> thay ngược lại)
    3. Dịch câu đã thay placeholder
    4. Ghép lại kết quả với entities đã dịch
    
    Args:
        text: Câu tiếng Nhật cần dịch
    
    Returns:
        str: Câu tiếng Anh đã dịch hoàn chỉnh
    """
    print("=" * 60)
    print("🚀 BẮT ĐẦU QUÁ TRÌNH DỊCH VỚI XỬ LÝ ENTITIES")
    print("=" * 60)
    print(f"📝 Input: {text}")
    print()
    
    try:
        # Bước 1: Thay thế entities bằng placeholders
        print("🔸 BƯỚC 1: Thay thế entities bằng placeholders")
        text_with_placeholders, ph2ent = replace_entity_and_map(text)
        print(f"📝 Text sau khi thay placeholder: {text_with_placeholders}")
        print(f"🗺️ Placeholder mapping: {ph2ent}")
        print()
        
        if not ph2ent:
            print("⚠️ Không tìm thấy entities, dịch trực tiếp...")
            return translate_text_simple(text)
        
        # Bước 2: Dịch entities
        print("🔸 BƯỚC 2: Dịch entities")
        translated_entities, entities_to_restore = translate_entities_with_fallback(ph2ent)
        print()
        
        # Bước 3: Thay ngược lại và dịch câu
        print("🔸 BƯỚC 3: Thay ngược lại entities không dịch được và dịch câu")
        final_text, translated_text, final_ph2ent = restore_entities_and_translate(
            text_with_placeholders, entities_to_restore, translated_entities
        )
        print()
        
        # Bước 4: Ghép lại kết quả
        print("🔸 BƯỚC 4: Ghép lại kết quả với entities đã dịch")
        final_result = merge_translation_with_entities(translated_text, final_ph2ent)
        print()
        
        print("=" * 60)
        print("✅ HOÀN THÀNH QUÁ TRÌNH DỊCH")
        print("=" * 60)
        print(f"📝 Kết quả cuối cùng: {final_result}")
        
        return final_result
        
    except Exception as e:
        print(f"❌ Lỗi trong quá trình dịch: {e}")
        print("🔄 Fallback: Dịch trực tiếp...")
        return translate_text_simple(text)


In [58]:
# Test với ví dụ từ yêu cầu
test_text = " まもなく、品川、品川。お出口は右側です。京浜東北線、山手線、京急線はお乗り換えです。本日もJR東日本をご利用くださいまして、ありがとうございました。"

print("🧪 TEST VỚI VÍ DỤ TỪ YÊU CẦU")
print("=" * 50)
result = translate_with_entity_handling(test_text)


🧪 TEST VỚI VÍ DỤ TỪ YÊU CẦU
🚀 BẮT ĐẦU QUÁ TRÌNH DỊCH VỚI XỬ LÝ ENTITIES
📝 Input:  まもなく、品川、品川。お出口は右側です。京浜東北線、山手線、京急線はお乗り換えです。本日もJR東日本をご利用くださいまして、ありがとうございました。

🔸 BƯỚC 1: Thay thế entities bằng placeholders
📝 Text sau khi thay placeholder:  ま[PH0]なく[PH1][PH2][PH1][PH2][PH3]お出口は右側です[PH3][PH4][PH1][PH5][PH1][PH6]はお乗り換えです[PH3]本日[PH0][PH7]をご利用くださいまして[PH1]ありがとうございました[PH3]
🗺️ Placeholder mapping: {'[PH0]': 'も', '[PH1]': '、', '[PH2]': '品川', '[PH3]': '。', '[PH4]': '京浜東北線', '[PH5]': '山手線', '[PH6]': '京急線', '[PH7]': 'JR東日本'}

🔸 BƯỚC 2: Dịch entities
🔄 Bắt đầu dịch entities...

📝 Xử lý [PH0] -> 'も'
  ❌ Không tìm được bản dịch cho 'も' -> sẽ thay ngược lại

📝 Xử lý [PH1] -> '、'
  ❌ Không tìm được bản dịch cho '、' -> sẽ thay ngược lại

📝 Xử lý [PH2] -> '品川'
  ✅ Tìm thấy trong CSV: '品川' -> 'Shinagawa'

📝 Xử lý [PH3] -> '。'
  ❌ Không tìm được bản dịch cho '。' -> sẽ thay ngược lại

📝 Xử lý [PH4] -> '京浜東北線'
  ✅ Tìm thấy trong CSV: '京浜東北線' -> 'Keihin–Tōhoku Line'

📝 Xử lý [PH5] -> '山手線'
  ✅ Tìm thấy trong CS